In [8]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap
import matplotlib.pyplot as plt
import skimage.io as io
import pylab

pylab.rcParams["figure.figsize"] = (16, 7.5)

import json
from json import encoder
import numpy as np

encoder.FLOAT_REPR = lambda o: format(o, ".3f")

In [9]:
blip2_model = "blip2-opt-2.7b"
clip_model = "ViT-32B"
leaked_feature_layer = "vit-base"

In [10]:
# 绘制所有Victim model指定的metric下的分布图
victim_models = {
    "ViT-16B": "ViT-16B",
    "ViT-32B": "ViT-32B",
    "RN50": "ResNet50",
    "RN101": "ResNet101",
    "mobilenetv2": "MobileNetV2",
    "mobilenetv3-small": "MobileNetV3-Small",
    "mobilenetv3-large": "MobileNetV3-Large",
}

leaked_feature_layers = {
    "ViT-16B": ["vit-base", "vit-no-proj"],
    "ViT-32B": ["vit-base", "vit-no-proj"],
    "RN50": [
        "resnet-base",
        "resnet-layer1",
        "resnet-layer2",
        "resnet-layer3",
        "resnet-layer4",
    ],
    "RN101": [
        "resnet-base",
        "resnet-layer1",
        "resnet-layer2",
        "resnet-layer3",
        "resnet-layer4",
    ],
    "mobilenetv2": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
    "mobilenetv3-small": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
    "mobilenetv3-large": [
        "mobilenet-base",
        "mobilenet-layer1",
        "mobilenet-layer-mid",
        "mobilenet-all-blocks",
    ],
}

In [11]:
metrics = [
    "Bleu_1",
    "Bleu_2",
    "Bleu_3",
    "Bleu_4",
    "METEOR",
    "ROUGE_L",
    "CIDEr",
    "SPICE",
]

In [12]:
blip2_model = "blip2-opt-2.7b"
dst_name = "imagenet"

In [ ]:
annFile = "/root/autodl-tmp/datasets/image_caption_generation/benjamin-paine/imagenet-1k-256x256/test/caption_test.json"
coco = COCO(annFile)

eval_val_dict = {}

for victim_model in victim_models:
    vic_val_dict = {}
    leaked_feature_layer = leaked_feature_layers[victim_model][0]
    resFile = f"/root/Image2CaptionAttack/processed_results/results_{dst_name}_{blip2_model}_{victim_model}_{leaked_feature_layer}.json"
    cocoRes = coco.loadRes(resFile)
    # create cocoEval object by taking coco and cocoRes
    cocoEval = COCOEvalCap(coco, cocoRes)

    # evaluate on a subset of images by setting
    # cocoEval.params['image_id'] = cocoRes.getImgIds()
    # please remove this line when evaluating the full validation set
    cocoEval.params["image_id"] = cocoRes.getImgIds()

    # evaluate results
    # SPICE will take a few minutes the first time, but speeds up due to caching
    cocoEval.evaluate()
    for metric in metrics:
        scores = [eva[metric] for eva in cocoEval.evalImgs]
        vic_val_dict[metric] = scores
    eval_val_dict[victim_model] = vic_val_dict

In [14]:
victim_model = "ViT-16B"

annFile = "/root/autodl-tmp/datasets/image_caption_generation/benjamin-paine/imagenet-1k-256x256/test/caption_test.json"
coco = COCO(annFile)

eval_val_dict_leaked_feature_layer = {}

for leaked_feature_layer in leaked_feature_layers[victim_model]:
    resFile = f"/root/Image2CaptionAttack/processed_results/results_{dst_name}_{blip2_model}_{victim_model}_{leaked_feature_layer}.json"
    cocoRes = coco.loadRes(resFile)
    # create cocoEval object by taking coco and cocoRes
    cocoEval = COCOEvalCap(coco, cocoRes)

    val_dict = {}

    # evaluate on a subset of images by setting
    # cocoEval.params['image_id'] = cocoRes.getImgIds()
    # please remove this line when evaluating the full validation set
    cocoEval.params["image_id"] = cocoRes.getImgIds()

    # evaluate results
    # SPICE will take a few minutes the first time, but speeds up due to caching
    cocoEval.evaluate()
    for metric in metrics:
        scores = [eva[metric] for eva in cocoEval.evalImgs]
        val_dict[metric] = scores

    eval_val_dict_leaked_feature_layer[leaked_feature_layer] = val_dict

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 12630 tokens at 89823.45 tokens per second.
PTBTokenizer tokenized 13055 tokens at 167217.46 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 10843, 'reflen': 10430, 'guess': [10843, 9943, 9043, 8144], 'correct': [4832, 1998, 877, 412]}
ratio: 1.039597315436142
Bleu_1: 0.446
Bleu_2: 0.299
Bleu_3: 0.206
Bleu_4: 0.145
computing METEOR score...
METEOR: 0.183
computing Rouge score...
ROUGE_L: 0.412
computing CIDEr score...
CIDEr: 1.182
computing SPICE score...


Parsing reference captions
Parsing test captions


SPICE evaluation took: 1.295 s
SPICE: 0.202
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 12630 tokens at 179862.21 tokens per second.
PTBTokenizer tokenized 13290 tokens at 201069.69 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 11098, 'reflen': 10430, 'guess': [11098, 10198, 9298, 8398], 'correct': [4514, 1648, 673, 300]}
ratio: 1.0640460210928988
Bleu_1: 0.407
Bleu_2: 0.256
Bleu_3: 0.168
Bleu_4: 0.114
computing METEOR score...
METEOR: 0.161
computing Rouge score...
ROUGE_L: 0.378
computing CIDEr score...
CIDEr: 0.865
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.4 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.4 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5

SPICE evaluation took: 15.02 s
SPICE: 0.161
